### Do NOT click RUN ALL... or else you will spend 20 minutes redefining the embeddings

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn
from Bio.Seq import Seq
np.random.seed(42)
from transformers import TFBertModel, BertTokenizer,BertConfig, AutoTokenizer, TFAutoModelForSequenceClassification, AdamWeightDecay
import re
np.random.seed(42)
tf.random.set_seed(42)
import pickle
import sys
import gc


Code for ProtBERT and BFD

Changes from previous version: I am implementing COVID-19 classification with ProtTrans (Rostlab models) starting with the BFD

# Data Formatting

In [2]:
df = pd.read_csv("../Data/CoV-AbDab_031022.csv")
df = df[["VHorVHH"]]
df = df[df["VHorVHH"].apply(lambda x: len(x) <= 138)]
df = df[(df.VHorVHH != 'ND')]
df
# df = df[["CDRH3"]]

,VHorVHH
5,QITLKESGPTLVKPTQTLTLTCKLSGFSVNTGGVGVGWIRQPPGKA...
32,QVQLVQSGAEVKKPGSSVKVSCKASGDTFNIYAINWVRQAPGQGLE...
33,QVQLVQSGAEVKKPGSSVKVSCKASGGTFNSYAITWVRQAPGQGLE...
34,QVQLVESGGGVVQPGRSLRLSCAASGFTFSTHGMHWVRQAPGKGLE...
35,QVQLVQSGAEVKKPGSSVKVSCKASGGTFRRYAISWVRQAPGQGLE...
...,...
11862,EVQVVESGGGLVKPGGSLRLSCAASGFTFSSYTMNWVRQAPGKGLE...
11863,QMQLVQSGPEVKRPGTSVKVSCEASGFTFSSSAILWVRQPRGQRLE...
11864,QVQLVESGGGLVKPGGSLRLSCAASGFTFSDYYMNWIRQAPGKGLE...
11865,EVQLVESGGGLVQPGGSLRLSCAASGFTFSRFAMHWVRQAPGKGLE...


In [3]:
dummy = []
head = []
with open("../Data/cAb-rep/cAb-Rep_heavy.nt.txt") as myfile:
    # count = 0
    for i in myfile:
        # if count <= 1:
        #     print(i)
        #     if i.find(">") == -1 & i.find("-") == -1:
        #         print(Seq.translate(i.strip()))
        #     count+=1
        dummy.append(i)
    np.random.shuffle(dummy)
    
    for i in dummy:
        if i.find(">") == -1 & i.find("-") == -1 & i.find("N") == -1: # These conditions must be met for a valid sequence, the longest was 141. However, there is no 141 sequence for COVID, the greatest is 138, so we go with that
            aa_sequence = Seq.translate(i.strip())
            if (len(aa_sequence) <= 138) & (len(aa_sequence) >= 100):
                head.append(aa_sequence)
                if len(head) >= 11415:
                    break
print(head[:5], len(head))
healthy_sequences = head

/Users/joseph/miniforge3/envs/test_env/lib/python3.10/site-packages/Bio/Seq.py:3482: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


['EVQLVQSGPEVKKPGSSVKVSCKASGGTFSNFAFSWVRQAPGQGLEWMGSVILHLGTSTYAQKFQGRVTITADESTSAAFMDLNALTSDDTAVYYCARVVAVPGRVPYWFDPWGQGTLVTVSS', 'TLSLTCAVYGGSFSGYYWSWIRQPPGKGLEWIGEINHSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARVPPTSTVTTLGDDYWGQGTLVTVSS', 'QVQLVQSGPEVKKPGASVRVSCKPSGYPFSNYGISWMRQAPGQGLEWMGWVNIDKGNTKYAQKFQDRVTMTTDTSSSTVYLELRSLRSDDTALYYCARERGGYRYGDYWGQGTLVIVSS', 'TLSLTCAVYGGSFSGYYWSWIRQPPGKGLEWIGEIKHSGSTNYIPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCASRAGAAAASWGQGTLVTVSS', 'SETLSLTCAVHGGSFSDYYWTWIRQPPGKGLEWIGEINHRGGTNYNPSLKSRLNILVDTSKSQFSLKLSSVTAADTAVYFCARERFILIRGLTKYYYYMDVWGKGTTVTVS'] 11415


In [4]:
del head
del myfile
del dummy
gc.collect()

0

In [5]:
covid_sequences = df.to_numpy()
covid_sequences = np.squeeze(covid_sequences)
np.random.shuffle(covid_sequences)
print(len(max(healthy_sequences, key=len)))
print(len(max(covid_sequences, key=len)))

138
138


In [6]:
# order from https://www.ncbi.nlm.nih.gov/Class/MLACourse/Modules/MolBioReview/iupac_aa_abbreviations.html

# Model

In [7]:
healthy_lables = [0] * 11415
covid_lables = [1] * 11415

In [8]:
X = np.concatenate((healthy_sequences, covid_sequences))
y = np.concatenate((healthy_lables, covid_lables))

In [9]:
X = X.tolist()
y = y.tolist()

In [10]:
del healthy_sequences
del covid_sequences
gc.collect()

0

## Splitting the data

Since the data we're loading isn't prepared for us as a machine learning dataset, we'll have to split the data into train and test sets ourselves! We can use sklearn's function for that:

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [12]:
del X
del y
gc.collect()

0

In [13]:
print(X_train[0])

EVQLVESGGGLIQPGGSLRLSCAASGFTVSSNYMSWVRQAPGKGLEWVSVIYSGGSTYYADSVKGRFTVSRDNSKNTLYLQMNSLRAEDTAVYYCARGGRYDYDVFDIWGQGTMVTVSS


In [14]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False)
# embedding_model = TFBertModel.from_pretrained("Rostlab/prot_bert_bfd", from_pt=True)
model = TFAutoModelForSequenceClassification.from_pretrained("Rostlab/prot_bert_bfd", num_labels=2, from_pt=True)

2023-01-02 16:02:57.630387: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-02 16:02:57.630520: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB



All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
def tokenize(sequence_list):
    sequences = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequence_list]
    sequences = [(" ".join(s)) for s in sequences]
    ids = tokenizer.batch_encode_plus(sequences, padding=True, return_tensors="tf")
    return ids

This looks good! We can see that our sequence has been converted into `input_ids`, which is the tokenized sequence, and an `attention_mask`. The attention mask handles the case when we have sequences of variable length - in those cases, the shorter sequences are padded with blank "padding" tokens, and the attention mask is padded with 0s to indicate that those tokens should be ignored by the model.

So now, let's tokenize our whole dataset. Note that we don't need to do anything with the labels, as they're already in the format we need.

In [16]:
train_tokenized = tokenize(X_train)
test_tokenized = tokenize(X_test)
val_tokenized = tokenize(X_val)

In [17]:
train_tokenized

{'input_ids': <tf.Tensor: shape=(18492, 140), dtype=int32, numpy=
array([[ 2,  9,  8, ...,  0,  0,  0],
       [ 2, 18, 10, ...,  0,  0,  0],
       [ 2,  7, 10, ...,  0,  0,  0],
       ...,
       [ 2, 18,  8, ...,  0,  0,  0],
       [ 2, 15,  5, ...,  0,  0,  0],
       [ 2, 18,  8, ...,  0,  0,  0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(18492, 140), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(18492, 140), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [18]:
from datasets import Dataset
train_dataset = Dataset.from_dict(train_tokenized)
test_dataset = Dataset.from_dict(test_tokenized)
val_dataset = Dataset.from_dict(val_tokenized)

train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 18492
})

This looks good, but we're missing our labels! Let's add those on as an extra column to the datasets.

In [19]:
train_dataset = train_dataset.add_column("labels", y_train) # train_labels = y_train
test_dataset = test_dataset.add_column("labels", y_test)
val_dataset = val_dataset.add_column("labels", y_val)

train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 18492
})

In [20]:
from transformers import AdamWeightDecay, TFAutoModelForSequenceClassification, AutoTokenizer

model = TFAutoModelForSequenceClassification.from_pretrained("Rostlab/prot_bert_bfd", num_labels=2, from_pt=True)

model.compile(optimizer=AdamWeightDecay(2e-5), metrics=["accuracy"])
model.summary()

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  419931136 
                                                                 
 dropout_183 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2050      
                                                                 
Total params: 419,933,186
Trainable params: 419,933,186
Non-trainable params: 0
_________________________________________________________________


In [21]:
tf_train_set = model.prepare_tf_dataset(
    train_dataset,
    batch_size=8,
    shuffle=True,
    tokenizer=tokenizer
)

tf_val_set = model.prepare_tf_dataset(
    val_dataset,
    batch_size=8,
    shuffle=False,
    tokenizer=tokenizer
)

tf_test_set = model.prepare_tf_dataset(
    test_dataset,
    batch_size=8,
    shuffle=False,
    tokenizer=tokenizer
)

In [22]:
model.fit(tf_train_set, validation_data=tf_val_set, epochs=3)

Epoch 1/3


2023-01-02 16:03:17.076655: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-02 16:03:29.290129: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2311/2311 [==============================] - ETA: 0s - loss: 0.5810 - accuracy: 0.6551

2023-01-02 16:34:03.114877: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2311/2311 [==============================] - 1979s 840ms/step - loss: 0.5810 - accuracy: 0.6551 - val_loss: 0.4931 - val_accuracy: 0.7392
Epoch 2/3
2311/2311 [==============================] - 2101s 907ms/step - loss: 0.6233 - accuracy: 0.6060 - val_loss: 0.6940 - val_accuracy: 0.4968
Epoch 3/3
2311/2311 [==============================] - 2272s 964ms/step - loss: 0.6945 - accuracy: 0.4979 - val_loss: 0.6931 - val_accuracy: 0.5032


In [ ]:
model.label2id = {"healthy": 0, "covid": 1}
model.id2label = {val: key for key, val in model.label2id.items()}

Now we can push it to the hub as simply as...

In [ ]:
model_name = "prot_berd_bfd"
finetuned_model_name = f"{model_name}-finetuned-healthy-covid-classification"

model.push_to_hub(finetuned_model_name)
tokenizer.push_to_hub(finetuned_model_name)

CommitInfo(commit_url='https://huggingface.co/josephyu12/antiberty-finetuned-healthy-covid-classification/commit/d8ece0542e3e08a8ecfce354d138be7c590ccdd9', commit_message='Upload tokenizer', commit_description='', oid='d8ece0542e3e08a8ecfce354d138be7c590ccdd9', pr_url=None, pr_revision=None, pr_num=None)